In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/train_skinCancer'
test_dir = '/content/drive/MyDrive/test_skinCancer'
train_benign_dr = '/content/drive/MyDrive/train_skinCancer/begnin'
train_malignant_dr = '/content/drive/MyDrive/train_skinCancer/malignent'
test_benign_dr = '/content/drive/MyDrive/test_skinCancer/begnin'
test_malignant_dr = '/content/drive/MyDrive/test_skinCancer/malignent'

In [ ]:
train_image_generator = ImageDataGenerator(rescale = 1./255)
test_image_generator = ImageDataGenerator(rescale = 1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size = 128,
                                                           directory = train_dir,
                                                           shuffle = True,
                                                           target_size = (224, 224),
                                                           class_mode = 'binary')

test_data_gen = test_image_generator.flow_from_directory(batch_size = 128,
                                                         directory = test_dir,
                                                         shuffle = True,
                                                         target_size = (224, 224),
                                                         class_mode = 'binary')

Found 500 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
train_data_gen.image_shape

(224, 224, 3)

In [ ]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(),
    layers.Dense(10, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss=losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 197136)            0         
                                                                 
 dense (Dense)               (None, 10)                1971370   
                                                                 
 dense_1 (Dense)             (None, 32)                352       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                        

In [ ]:
total_train_size = len(os.listdir(train_benign_dr)) + len(os.listdir(train_malignant_dr))
total_test_size = len(os.listdir(test_benign_dr)) + len(os.listdir(test_malignant_dr))

In [ ]:
model.fit(train_data_gen,
          batch_size = 128,
          epochs = 20,
          steps_per_epoch = total_train_size // 128,
          validation_data = test_data_gen,
          validation_steps = total_test_size // 128)

Epoch 1/20
3/3 [==============================] - 16s 5s/step - loss: 5.2453 - accuracy: 0.4866 - val_loss: 1.3609 - val_accuracy: 0.5312
Epoch 2/20
3/3 [==============================] - 14s 5s/step - loss: 1.0437 - accuracy: 0.5081 - val_loss: 0.6870 - val_accuracy: 0.5156
Epoch 3/20
3/3 [==============================] - 14s 5s/step - loss: 0.7285 - accuracy: 0.4704 - val_loss: 0.7282 - val_accuracy: 0.4844
Epoch 4/20
3/3 [==============================] - 15s 5s/step - loss: 0.6818 - accuracy: 0.5941 - val_loss: 0.7891 - val_accuracy: 0.4609
Epoch 5/20
3/3 [==============================] - 14s 5s/step - loss: 0.6925 - accuracy: 0.5188 - val_loss: 0.7158 - val_accuracy: 0.5547
Epoch 6/20
3/3 [==============================] - 15s 6s/step - loss: 0.5862 - accuracy: 0.6989 - val_loss: 0.6606 - val_accuracy: 0.5781
Epoch 7/20
3/3 [==============================] - 15s 5s/step - loss: 0.5293 - accuracy: 0.7796 - val_loss: 0.6866 - val_accuracy: 0.6797
Epoch 8/20
3/3 [==================

In [ ]:
def result(predictions):
  if predictions > 0.5:
    print("Malignant")
  else:
    print("benign")

In [ ]:
from PIL import Image

image_path = '/content/drive/MyDrive/test_skinCancer/begnin/melanoma_9614.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))
image = np.array(image) / 255.0

image = image.reshape((1, 224, 224, 3))

predictions = model.predict(image)
print(predictions)
result(predictions)

In [ ]:
from PIL import Image

image_path = '/content/drive/MyDrive/test_skinCancer/begnin/melanoma_9719.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))
image = np.array(image) / 255.0

image = image.reshape((1, 224, 224, 3))

predictions = model.predict(image)
print(predictions)
result(predictions)

In [ ]:
from PIL import Image

image_path = '/content/drive/MyDrive/test_skinCancer/malignent/melanoma_10202.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))
image = np.array(image) / 255.0

image = image.reshape((1, 224, 224, 3))

predictions = model.predict(image)
print(predictions)
result(predictions)

In [ ]:
tf.keras.models.save_model(model,'my_model.hdf5')

In [ ]:
! pip install streamlit -q

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import streamlit as st


@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('/content/my_model.hdf5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Skin cancer prediction
         """
         )

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):

        size = (224, 224)
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.

        img_reshape = img[np.newaxis,...]

        prediction = model.predict(img_reshape)

        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    st.write(predictions)
    st.write(score)
    class_names = ['Malignent','bengin']
    print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

In [ ]:
import tensorflow as tf
tf.keras.models.save_model(model,'my_model.hdf5')